<a href="https://colab.research.google.com/github/infinite-darkness108/EEG_analysis/blob/main/ML_ON_EEG_MNE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
!pip install mne -U

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

import mne
from mne.datasets import sample
from mne.decoding import (SlidingEstimator, GeneralizingEstimator, Scaler,
                          cross_val_multiscore, LinearModel, get_coef,
                          Vectorizer, CSP)

data_path = sample.data_path()

subjects_dir = data_path / 'subjects'
meg_path = data_path / 'MEG' / 'sample'
raw_fname = meg_path / 'sample_audvis_filt-0-40_raw.fif'
tmin, tmax = -0.200, 0.500
event_id = {'Auditory/Left': 1, 'Visual/Left': 3}  # just use two
raw = mne.io.read_raw_fif(raw_fname)
raw.pick_types(meg='grad', stim=True, eog=True, exclude=())

# The subsequent decoding analyses only capture evoked responses, so we can
# low-pass the MEG data. Usually a value more like 40 Hz would be used,
# but here low-pass at 20 so we can more heavily decimate, and allow
# the example to run faster. The 2 Hz high-pass helps improve CSP.
raw.load_data().filter(2, 20)
events = mne.find_events(raw, 'STI 014')

# Set up bad channels (modify to your needs)
raw.info['bads'] += ['MEG 2443']  # bads + 2 more

# Read epochs
epochs = mne.Epochs(raw, events, event_id, tmin, tmax, proj=True,
                    picks=('grad', 'eog'), baseline=(None, 0.), preload=True,
                    reject=dict(grad=4000e-13, eog=150e-6), decim=3,
                    verbose='error')
epochs.pick_types(meg=True, exclude='bads')  # remove stim and EOG
del raw

X = epochs.get_data()  # MEG signals: n_epochs, n_meg_channels, n_times
y = epochs.events[:, 2]  # target: auditory left vs visual left

Opening raw data file /root/mne_data/MNE-sample-data/MEG/sample/sample_audvis_filt-0-40_raw.fif...
    Read a total of 4 projection items:
        PCA-v1 (1 x 102)  idle
        PCA-v2 (1 x 102)  idle
        PCA-v3 (1 x 102)  idle
        Average EEG reference (1 x 60)  idle
    Range : 6450 ... 48149 =     42.956 ...   320.665 secs
Ready.
Removing projector <Projection | PCA-v1, active : False, n_channels : 102>
Removing projector <Projection | PCA-v2, active : False, n_channels : 102>
Removing projector <Projection | PCA-v3, active : False, n_channels : 102>
Removing projector <Projection | Average EEG reference, active : False, n_channels : 60>
Reading 0 ... 41699  =      0.000 ...   277.709 secs...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 2 - 20 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s


319 events found
Event IDs: [ 1  2  3  4  5 32]


[Parallel(n_jobs=1)]: Done 204 out of 204 | elapsed:    0.4s finished


In [ ]:
epochs

Number of events,123
Events,Auditory/Left: 56Visual/Left: 67
Time range,-0.200 – 0.499 sec
Baseline,-0.200 – 0.000 sec


In [ ]:
X

array([[[-6.56767535e-13,  6.02433416e-13,  2.80465629e-12, ...,
          1.50546377e-12,  3.30228036e-12, -1.10558462e-12],
        [-1.03219337e-12, -2.87528351e-13, -4.49899236e-13, ...,
         -1.54581553e-12, -1.75118751e-12, -1.70937069e-12],
        [ 5.28657741e-13,  2.92813496e-12,  2.64485502e-12, ...,
         -2.18572451e-12, -2.53676446e-12, -5.55858765e-12],
        ...,
        [ 2.15028775e-12, -9.75114729e-14,  5.13316079e-13, ...,
         -1.69706401e-12, -2.78608377e-12, -8.16944053e-13],
        [-1.56443393e-12, -4.16787315e-12, -8.30542219e-13, ...,
          2.97783509e-12, -3.31815513e-12,  1.37551603e-12],
        [ 3.11463420e-12, -1.26726702e-12, -4.02141900e-12, ...,
         -5.43392733e-12, -3.30273452e-12,  8.57769516e-13]],

       [[ 1.55423440e-12,  1.34285730e-12, -3.58504782e-12, ...,
          2.62644652e-13,  1.16829653e-12,  3.05261987e-12],
        [-3.32177750e-12, -6.53494782e-13,  1.38674670e-13, ...,
         -1.84204336e-12, -1.04439831e

In [ ]:
y

array([3, 1, 3, 1, 3, 1, 3, 1, 3, 1, 3, 1, 3, 1, 3, 1, 3, 1, 3, 1, 3, 1,
       3, 1, 3, 1, 3, 3, 1, 3, 1, 3, 1, 3, 1, 1, 3, 1, 3, 1, 3, 3, 1, 3,
       1, 3, 1, 3, 3, 1, 3, 1, 3, 1, 3, 1, 3, 1, 1, 3, 3, 1, 3, 1, 3, 1,
       3, 3, 1, 3, 1, 3, 1, 3, 1, 3, 3, 3, 1, 3, 3, 3, 3, 1, 3, 1, 3, 1,
       1, 3, 1, 3, 1, 3, 1, 3, 1, 3, 3, 1, 3, 3, 1, 3, 3, 1, 3, 1, 3, 3,
       3, 1, 3, 1, 3, 1, 3, 1, 3, 1, 1, 3, 1])

In [ ]:
# Uses all MEG sensors and time points as separate classification
# features, so the resulting filters used are spatio-temporal
clf = make_pipeline(
    Scaler(epochs.info),
    Vectorizer(),
    LogisticRegression(solver='liblinear')  # liblinear is faster than lbfgs
)

scores = cross_val_multiscore(clf, X, y, cv=5, n_jobs=None)

# Mean scores across cross-validation splits
score = np.mean(scores, axis=0)
print('Spatio-temporal: %0.1f%%' % (100 * score,))

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.5s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.7s remaining:    0.0s


Spatio-temporal: 99.2%


[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.8s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    1.0s finished


In [ ]:
X_2d = X.reshape(X.shape[0], -1)
clf.fit(X_2d, y)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('linearmodel',
                 LinearModel({'model': LogisticRegression(solver='liblinear')}))])

In [ ]:
test_idx =20

In [ ]:
X_2d[test_idx]

array([-1.98370117e-12,  9.13284551e-13, -1.22589613e-13, ...,
       -5.69365521e-12, -1.26314208e-11, -1.92145685e-12])

In [ ]:
y[test_idx]

# The output ↓ is corrctly predicted or not is the question.

3

In [ ]:
clf.predict(X_2d[test_idx].reshape(1,-1))

array([3])